In [2]:
import xarray as xr
import rioxarray as rio
import pandas as pd
import numpy as np
from glob import glob
from itertools import product
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent) + "/code/")
from utils import *

In [3]:
step0 = rio.open_rasterio('../data/biomass/liu_et_al_2015/Global_annual_mean_ABC_lc2001_1993_2012_20150331.nc',masked=True)['Aboveground Biomass Carbon']

# set the coordinatess to the same as the other datasets
step0 = xr.DataArray(data=step0.values.swapaxes(2,1)[:,:,::-1],
                    coords=[step0['time'].values,np.linspace(89.875,-89.875,720),np.linspace(-179.875,179.875,1440)],
                    dims=['time','y','x'])
step0 = step0.rio.write_crs(4326)
step0['time'] = step0['time'].astype(int)

step0_rate = (100*step0*calc_area(step0)).sum(dim=['x','y']).diff(dim='time').mean().values/1e15
biomass_source = 'liu_biomass'


In [4]:
LVOD_data = xr.open_dataset('../data/biomass/LVOD/AGC_vod_annual_NOAA_Trend_corrected_lat_lon_merged.nc')['AGC_ASC_DESC']
LVOD_data.rio.write_crs(4326,inplace=True);

LVOD_data_ASC_DESC = LVOD_data[0,:,:,:]
LVOD_data_ASC_DESC_max = LVOD_data[1,:,:,:]
LVOD_data_ASC_DESC_min = LVOD_data[2,:,:,:]

step0_rate = (100*LVOD_data_ASC_DESC*calc_area(LVOD_data_ASC_DESC)).sum(dim=['x','y']).diff(dim='time').mean().values/1e15
biomass_source = 'LVOD'#'liu_biomass'

In [5]:

fsm = ['xu','CCI','song']
step1_rates = {}
for f in fsm:
    step1 = xr.open_dataarray(f'../results/01_split_forest_nonforest/{biomass_source}_{f}.nc')
    step1_rate = (100*step1*calc_area(step1)).sum(dim=['x','y','landcover']).diff(dim='time').mean().values/1e15
    step1_rates[f] = step1_rate
    print(f'step 1 rate for {f}: {step1_rate:.4f} PgC/yr and difference from step 0 is: {step1_rate-step0_rate:.4f}')

step 1 rate for xu: 3.1751 PgC/yr and difference from step 0 is: -0.0000
step 1 rate for CCI: 3.1751 PgC/yr and difference from step 0 is: 0.0000
step 1 rate for song: 3.1751 PgC/yr and difference from step 0 is: 0.0000


In [6]:

fsm = ['xu','CCI','song']
RMFS = ['huang','ma']
step2_rates = {}
for f,r in product(fsm,RMFS):
    step2 = xr.open_dataarray(f'../results/02_convert_AGB_TB/{biomass_source}_{f}_{r}_TB.nc')
    step2_rate = (100*step2*calc_area(step2)).sum(dim=['x','y','landcover']).diff(dim='time').mean().values/1e15
    step2_rates['_'.join([f,r])] = step2_rate
    print(f'step 2 rate for {f},{r}: {step2_rate:.4f} PgC/yr; the ratio between AGB and TB is: {step2_rate/step1_rates[f]:.2f}')

step 2 rate for xu,huang: 3.7586 PgC/yr; the ratio between AGB and TB is: 1.18
step 2 rate for xu,ma: 4.5014 PgC/yr; the ratio between AGB and TB is: 1.42
step 2 rate for CCI,huang: 3.7586 PgC/yr; the ratio between AGB and TB is: 1.18
step 2 rate for CCI,ma: 4.5014 PgC/yr; the ratio between AGB and TB is: 1.42
step 2 rate for song,huang: 3.7586 PgC/yr; the ratio between AGB and TB is: 1.18
step 2 rate for song,ma: 4.5014 PgC/yr; the ratio between AGB and TB is: 1.42


In [7]:
fsm = ['xu','CCI','song']
RMFS = ['huang','ma']
step3_rates = {}
for f,r in product(fsm,RMFS):
    step3 = pd.read_csv(f'../results/03_aggregate_regions/{biomass_source}_{f}_{r}_TB_regions.csv',index_col=[0,1])
    step3_rate = step3.sum().diff().mean()/1e15
    step3_rates['_'.join([f,r])] = step3_rate
    print(f'step 3 rate for {f},{r}: {step2_rate:.4f} PgC/yr; the difference from step 2 is: {step3_rate-step2_rates["_".join([f,r])]:.5f}')

step 3 rate for xu,huang: 4.5014 PgC/yr; the difference from step 2 is: -0.00000
step 3 rate for xu,ma: 4.5014 PgC/yr; the difference from step 2 is: 0.00000
step 3 rate for CCI,huang: 4.5014 PgC/yr; the difference from step 2 is: -0.00000
step 3 rate for CCI,ma: 4.5014 PgC/yr; the difference from step 2 is: 0.00000
step 3 rate for song,huang: 4.5014 PgC/yr; the difference from step 2 is: -0.00000
step 3 rate for song,ma: 4.5014 PgC/yr; the difference from step 2 is: -0.00000


In [8]:
fsm = ['xu','CCI','song']
RMFS = ['huang','ma']
step4_rates = {}
for f,r in product(fsm,RMFS):
    step4 = pd.read_csv('../results/04_temporal_harmonization/harmonized_biomass_data.csv',index_col=[0,1,2,3])
    step4_rate = step4.loc[pd.IndexSlice[biomass_source.split('_biomass')[0],f'{biomass_source}_{f}_{r}_TB_regions',:,:],:].mean(axis=1).sum()/1e15
    if biomass_source == 'LVOD':
        step4_rate = step4_rate*9
    step4_rates['_'.join([f,r])] = step4_rate
    print(f'step 4 rate for {f},{r}: {step2_rate:.4f} PgC/yr; the difference from step 3 is: {step4_rate-step3_rates["_".join([f,r])]:.5f}')
# print(f'step 4 rate: {step4_rate:.4f} PgC/yr')

step 4 rate for xu,huang: 4.5014 PgC/yr; the difference from step 3 is: -0.00000
step 4 rate for xu,ma: 4.5014 PgC/yr; the difference from step 3 is: -0.00000
step 4 rate for CCI,huang: 4.5014 PgC/yr; the difference from step 3 is: 0.00000
step 4 rate for CCI,ma: 4.5014 PgC/yr; the difference from step 3 is: 0.00000
step 4 rate for song,huang: 4.5014 PgC/yr; the difference from step 3 is: -0.00000
step 4 rate for song,ma: 4.5014 PgC/yr; the difference from step 3 is: -0.00000
